# Cross entropy reliability algorithm with openturns

Source of the algorithm : J. Morio & M. Balesdent, Estimation of Rare Event Probabilities in Complex Aerospace and Other Systems, A Practical Approach, Elsevier, 2015


The theory is given for a failure event defined as $\phi(\mathbf{X})>S$ with $\mathbf{X}$ a random vector following a joint PDF $h_0$, $S$ a threshold and $\phi$ a limit state function, without loss of generality.

The IS probability estimate by Importance Sampling $\widehat{P}^{IS}$ is given by 
\begin{equation}
\widehat{P}^{IS}=\frac{1}{N} \sum_{i=1}^{N} {\bf 1}_{\phi(\mathbf{X}_i)>S} \frac{h_0(\mathbf{X}_i)}{h(\mathbf{X}_i)}.
\label{ISeq}
\end{equation}

It is well-known that the optimal density minimizing the variance of the estimator $h_{opt}$ is  defined as
\begin{equation}
h_{opt}=\frac{{\mathbf 1}_{\phi(x)>S}h_0}{P}.
\label{opt}
\end{equation}


with $P$ the failure probability and is inaccessible in practice since this probability is unknown. 

Let us define $h_\lambda$, a family of densities indexed by a parameter $\lambda\in \Delta$ where $\Delta$ is the multidimensional space of pdf parameters. The parameter $\lambda$ is, for instance, the mean and the variance in the case of Gaussian densities. The objective of importance sampling with Cross-Entropy (CE) is to determine the parameter $\lambda_{opt}$ that minimizes the Kullback-Leibler divergence between $h_{\lambda_{opt}}$ and $h_{opt}$. The value of $\lambda_{opt}$ is thus obtained as follows
\begin{equation}
\lambda_{opt}= \underset{\lambda}{\operatorname{argmin}}\left\{ D(h_{opt},h_\lambda) \right\},
\label{opti}
\end{equation}
where $D$ is the Kullback-Leibler divergence defined between two densities $p$ and $q$ by 
\begin{equation}
D(q,p)=\int_{\mathbb{R}^d} q(x) \ln(q(x))dx- \int_{\mathbb{R}^d} q(x) \ln(p(x))dx.
\end{equation} 

Determining the parameter $\lambda_{opt}$ is not obvious since it depends on the unknown pdf $h_{opt}$. In fact, it can be shown that it is equivalent to solve
\begin{equation}
\lambda_{opt}=\underset{\lambda}{\operatorname{argmax}}\left\{  E\left[{\bf 1}_{\phi(\mathbf{X})>S} \ln \left(h_\lambda(\mathbf{X}) \right)\right] \right\},
\label{CE1}
\end{equation}
where $E$ defines the expectation operator. In fact, one does not focus directly on the preceding equation  but proceeds iteratively to estimate $\lambda_{opt}$ with an iterative sequence of thresholds,
\begin{equation}
\gamma_0<\gamma_1 < \gamma_2 < ... <\gamma_k<...\leq S,
\end{equation}
chosen adaptively using quantile definition. At iteration $k$, the value $\lambda_{k-1}$ is available and one maximizes in practice
\begin{equation}
\lambda_k=\underset{\lambda}{\operatorname{argmax}} \frac{1}{N} \sum_{i=1}^N {\bf 1}_{\phi(\mathbf{X}_i)>\gamma_k} \frac{h_0(\mathbf{X}_i)}{h_{\lambda_{k-1}}(\mathbf{X}_i)} \ln (h_{\lambda}(\mathbf{X}_i)).
\end{equation}
where the samples $\mathbf{X}_1,...,\mathbf{X}_N$ are generated with $h_{\lambda_{k-1}}$.
The probability $\widehat{P}^{CE}$ is then estimated with importance sampling at the last iteration. The Cross-Entropy optimization algorithm for importance sampling density is



1. $k=1$, define $h_{\lambda_0}=h_0$ and set $\rho \in [0,1]$
2. Generate the population $\mathbf{X}_1,\cdots,\mathbf{X}_N$ according to the pdf $h_{\lambda_{k-1}}$ and apply the function $\phi$ in order to have $Y_1=\phi(\mathbf{X}_1),\ldots,Y_N=\phi(\mathbf{X}_N)$
3. Compute the empirical $\rho$-quantile $q_k=\min(S, Y_{\left\lfloor\rho N\right\rfloor})$, where $\lfloor a\rfloor$ denotes the largest integer that is smaller than or equal to $a$
4. Optimize the parameters of the auxiliary pdf family as $\lambda_k=\underset{\lambda}{\operatorname{argmax}}\left\{\frac{1}{N}\displaystyle \sum_{i=1}^N\left[1_{\phi(\mathbf{X}_i)>q_k} \frac{h_0(\mathbf{X}_i)}{h_{\lambda_{k-1}}(\mathbf{X}_i)}ln\left[h_\lambda(\mathbf{X}_i)\right]\right]\right\}$
5. If $q_k<S$, $k\leftarrow k+1$, go to Step 2
6. Estimate the probability $\widehat{P}^{CE}(\phi(\mathbf{\mathbf{X}}>S))=\frac{1}{N}\displaystyle \sum_{i=1}^{N} 1_{\phi(\mathbf{X}_i)>S} \frac{h_0(\mathbf{X}_i)}{h_{\lambda_k-1}(\mathbf{X}_i)}$


This algorithm is implemented in the following class.

In [1]:
import openturns as ot
import math as m
from  CrossEntropyAlgorithm import CrossEntropyAlgorithm

## Numerical experiments

http://openturns.github.io/openturns/master/examples/reliability_sensitivity/estimate_probability_monte_carlo.html


We consider a simple beam stressed by a traction load F at both sides.

The geometry is supposed to be deterministic; the diameter D is equal to:

$D=0.02 \textrm{ (m)}.$

By definition, the yield stress is the load divided by the surface. Since the surface is \pi D^2/4, the stress is:

$S = \frac{F}{\pi D^2/4}.$

Failure occurs when the beam plastifies, i.e. when the axial stress gets larger than the yield stress:

$R - \frac{F}{\pi D^2/4} \leq 0$

where R is the strength.

Therefore, the limit state function G is:

$G(R,F) = R - \frac{F}{\pi D^2/4},$

for any R,F\in\mathbb{R}.

The value of the parameter D is such that:

$D^2/4 = 10^{-4},$

which leads to the equation:

$G(R,F) = R - \frac{F}{10^{-4} \pi}.$

with

$R \sim LogNormal(\mu_R=3\times 10^6, \sigma_R=3\times 10^5) [Pa]$

$F \sim Normal(\mu_F=750, \sigma_F=50) [N]$

In [2]:
#Creation of the event
distribution_R = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_F = ot.Normal(75e3, 5e3)
marginals = [distribution_R, distribution_F]
distribution = ot.ComposedDistribution(marginals)

# create the model
model = ot.SymbolicFunction(['R', 'F'], ['R-F/(pi_*100.0)'])

#create the event 
vect = ot.RandomVector(distribution)
G = ot.CompositeRandomVector(model, vect)
event = ot.ThresholdEvent(G, ot.Less(), 0.0)

In [3]:
#Determination of reference probability
#MonteCarlo experiment
n_MC = 1e6

# create a Monte Carlo algorithm
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)
algo.setMaximumOuterSampling(int(n_MC))
algo.setMaximumCoefficientOfVariation(0.01)
algo.run()
# retrieve results
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)

Pf= 0.0294834174762001


In [4]:
# Parameters of the distribution  (warning : native parameters)
distribution.getParameter()

class=Point name=Unnamed dimension=5 values=[5.69881,0.0997513,0,75000,5000]

## Cross entropy estimation with full parameter optimization

In [5]:
# Hyperparameters of the algorithm
n_IS= 2500 # Number of samples at each iteration
rho_quantile= 25 # Quantile determining the percentage of failure samples in the current population 


## Definition of auxiliary distribution
distribution_margin1 = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_margin2 = ot.Normal(75e3, 5e3)
aux_marginals = [distribution_margin1, distribution_margin2]
aux_distribution = ot.ComposedDistribution(aux_marginals)


## Definition of parameters to be optimized
active_parameters = [True,True,True,True,True] # active parameters from the auxiliary distribution which will be optimized
### WARNING : native parameters of distribution have to be considered


bounds = ot.Interval([3,0.09,0.,50e3,2e3], # bounds on the active parameters
                     [7,0.5,0.5,100e3,10e3])

initial_theta= [5.70,0.1,0.,75e3,5e3] # initial value of the active parameters

verbose = True # verbosity parameter, if true, the values of auxiliary parameters and the current threshold will be displayed

## Definition of the algorithm
CE_1 = CrossEntropyAlgorithm(event,n_IS,rho_quantile,distribution,active_parameters,bounds,initial_theta,verbose)

# Run of the algorithm
CE_1.run()
CE_1results = CE_1.getResult()
print('Probability of failure:',CE_1results.getProbabilityEstimate())


theta | current threshold
[5.5036,0.09,0.5,77770.2,9836.89] | 38.172025248957866
[5.65353,0.424957,0.00964724,83470.6,9932.45] | -25.36969524711087
0.029083411611290275
Probability of failure: 0.029083411611290275


In [8]:
# Found optimal auxiliary density
CE_1results.getAuxiliaryDensity()

class=ComposedDistribution name=ComposedDistribution dimension=2 copula=class=IndependentCopula name=IndependentCopula dimension=2 marginal[0]=class=LogNormal name=LogNormal dimension=1 muLog=5.5036 sigmaLog=0.09 gamma=0.5 marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[77770.2] sigma=class=Point name=Unnamed dimension=1 values=[9836.89] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1]

## Cross entropy estimation with partial parameter optimization (only the means of the margins, all the others as considered as their reference values)

In [11]:
# Hyperparameters of the algorithm
n_IS= 2500 # Number of samples at each iteration
rho_quantile= 25 # Quantile determining the percentage of failure samples in the current population 


## Definition of auxiliary distribution
distribution_margin1 = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_margin2 = ot.Normal(75e3, 5e3)
aux_marginals = [distribution_margin1, distribution_margin2]
aux_distribution = ot.ComposedDistribution(aux_marginals)


## Definition of parameters to be optimized
active_parameters = [True,False,False,True,False] # active parameters from the auxiliary distribution which will be optimized
### WARNING : native parameters of distribution have to be considered
bounds =  ot.Interval([3,50e3], # bounds on the active parameters
                       [7,100e3])

initial_theta= [5.70,75e3] # initial value of the active parameters

verbose = True# verbosity parameter, if true, the values of auxiliary parameters and the current threshold will be displayed

## Definition of the algorithm
CE_2 = CrossEntropyAlgorithm(event,n_IS,rho_quantile,distribution,active_parameters,bounds,initial_theta,verbose)

# Run of the algorithm
CE_2.run()
CE_2results = CE_2.getResult()
print('Probability of failure:',CE_2results.getProbabilityEstimate())

theta | current threshold
[5.72496,77893.1] | 38.38778597614599
[5.71295,81827.3] | 37.88114215855242
[5.71295,81827.3] | 21.22861007913673
[5.71295,81827.3] | 20.331365412610083
[5.71295,81827.3] | 20.205546550169544
[5.68248,80810.4] | 18.450586562334266
[5.47146,78067.9] | 16.239282906539742
[5.41648,79684.4] | -28.94654371748183
0.030238614762599675
Probability of failure: 0.030238614762599675


In [13]:
# Found optimal auxiliary density
CE_2results.getAuxiliaryDensity()

class=ComposedDistribution name=ComposedDistribution dimension=2 copula=class=IndependentCopula name=IndependentCopula dimension=2 marginal[0]=class=LogNormal name=LogNormal dimension=1 muLog=5.47146 sigmaLog=0.0997513 gamma=0 marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[78067.9] sigma=class=Point name=Unnamed dimension=1 values=[5000] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1]

In [12]:
# Found CE Samples at the last iteration
CE_2results.getSamples()

class=Sample name=ComposedDistribution implementation=class=SampleImplementation name=ComposedDistribution size=2500 dimension=2 description=[X0,X1] data=[[276.364,79138.3],[205.193,77774.9],[238.964,85730.5],...,[232.164,75258.2],[236.996,81390.1],[222.456,75988.6]]